# Inundation Visualization with Relative Elevation Models (REM)

![Inundation of Troublesome Creek](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/lab-03-work.jpg)   
*Inundation of Troublesome Creek*


In [1]:
############### Imports :: DO NOT EDIT ###############

# Import ArcGIS package
import arcpy
# Subprocess allows us to issue commands on the command line
import subprocess
# Get current local time
import datetime

In [2]:
######### MODIFY THIS CELL: change to variables to match your local environment ######### 

# Where is your root GIS folder?
r = 'c:\\BoydsGIS'

# What is your project's folder name?
p = 'inundation_'

# What is your stream flowline layer name?
b = 'NHDFlowline_tc'

In [3]:
############### Global variables and functions :: DO NOT EDIT ###############

# Set up our environment
arcpy.env.workspace = f'{r}\\{p}\\{p}.gdb'
db = f'{r}\\{p}\\{p}.gdb'
ky = arcpy.SpatialReference(3089)
arcpy.env.outputCoordinateSystem = ky
arcpy.env.overwriteOutput = True
dsm = 'https://kyraster.ky.gov/arcgis/rest/services/ElevationServices/Ky_DSM_First_Return_5FT_Phase1/ImageServer'
dem = 'https://kyraster.ky.gov/arcgis/rest/services/ElevationServices/Ky_DEM_KYAPED_5FT/ImageServer'
img = 'https://kyraster.ky.gov/arcgis/rest/services/ImageServices/Ky_KYAPED_Phase1_6IN/ImageServer'


def listContents(root, folder):
    """List contents of  a directory. 'root' parameter is GIS space and 'folder' is any inside GIS space"""
    downloads = f'{root}\\{folder}'
    completed = subprocess.run(f'dir {downloads}', shell=True, stdout=subprocess.PIPE)
    print(completed.stdout.decode('UTF-8'))
    print(f'Feature classes in geodatabase {arcpy.ListFeatureClasses()}')
    print("Finished processing", datetime.datetime.now())

def selectRiver(name):
    """Select Kentucky river by 'name' as a string. No need to enter 'River' or 'North', e.g. """
    where = f'"GNIS_Name" like \'%{name}%\''
    print(f'Using {where} to match all stream names that include {name}...')
    arcpy.Select_analysis(b, "candidate", where)
    print('Edit the section of the stream that you want to create the REM. \nThen select that section using the Select tool.')
    print("Finished processing", datetime.datetime.now())
    
def processRiver(distance, buffer):
    """Create vector analysis layers. 'distance' in feet between sample points, 'buffer' distance in feet """
    arcpy.env.addOutputsToMap = False
    select = input("Do you have a selection to process? Yes or No ")
    if 'y' in select.lower():
        print(f'AOI will be {buffer} feet from flowline. Placing points every {distance} feet along flowline...')
        arcpy.env.extent = "MAXOF"
        arcpy.management.UnsplitLine("candidate", "candidate_unsplit")
        arcpy.analysis.Buffer('candidate_unsplit', 'river_buffer', buffer)
        arcpy.management.GeneratePointsAlongLines('candidate_unsplit', 'river_pts', "DISTANCE", distance)
    else:
        print("Select a portion of a river")
    arcpy.env.addOutputsToMap = True
    print("Finished processing", datetime.datetime.now())
        
def processREM(buffer):
    """Create REM using IDW interpolation from river sample points. 'buffer' distance in feet """
    arcpy.env.addOutputsToMap = False
    print('Clipping DEM to AOI...')
    arcpy.management.Clip(dem, "#", 'dem', 'river_buffer', '#', True)
    print(f'Sampling ground elevation along flowline...')
    arcpy.sa.ExtractValuesToPoints('river_pts', 'dem', 'river_pts_elev')
    arcpy.env.extent = "river_buffer"
    print(f'Interpolating stream height...')
    arcpy.ddd.Idw("river_pts_elev", "RASTERVALU", "idw", 5, 2, f"FIXED {buffer}", None)
    idw = arcpy.sa.Raster('idw')
    dem_o = arcpy.sa.Raster('dem')
    print(f'Calculating REM on ground surface...')
    out = (dem_o - idw)
    out.save(f'rem')
    dsm_o = arcpy.sa.Raster(dsm)
    print(f'Calculating REM on above ground surface...')
    out = (dsm_o - idw)
    out.save(f'rem_dsm')
    arcpy.env.addOutputsToMap = True
    print("Finished processing", datetime.datetime.now())

def processInundation(height):
    """Create inundation models using REMs. 'height' is elevation above stream in feet """
    arcpy.env.addOutputsToMap = False
    print(f'Calculating extent of innudation to {height} feet above stream level for ground...')
    rem = arcpy.sa.Raster(f'{db}\\rem')
    a = arcpy.sa.Con(rem <= 0, 0, rem)
    b = arcpy.sa.Con(a <= height, a)
    c = arcpy.sa.Abs(b - height)
    c.save(f'inun_dem_{height}ft')
    print(f'Calculating extent of innudation to {height} feet above stream level for above ground features...')
    rem_dsm = arcpy.sa.Raster(f'{db}\\rem_dsm')
    d = arcpy.sa.Con(rem_dsm <= 0, 0, rem_dsm)
    e = arcpy.sa.Con(d <= height, d, height)
    f = arcpy.sa.Abs(e - height)
    g = f + rem_dsm
    g.save(f'inun_surface_{height}ft')
    arcpy.ddd.HillShade(g, f'inun_hillshade_{height}ft')
    ee = arcpy.sa.Con(d <= height, d)
    ff = arcpy.sa.Abs(ee - height)
    ff.save(f'inun_dsm_{height}ft')
    arcpy.env.addOutputsToMap = True
    print("Finished processing", datetime.datetime.now())


In [4]:
# Let's see what you have in your project folder and geodatabase
listContents(r, p)

 Volume in drive C has no label.
 Volume Serial Number is 0AA6-9A6D

 Directory of c:\BoydsGIS\inundation_

10/19/2022  01:28 PM    <DIR>          .
10/19/2022  01:28 PM    <DIR>          ..
10/19/2022  01:28 PM    <DIR>          .backups
10/12/2022  11:07 AM    <DIR>          .ipynb_checkpoints
10/14/2022  12:01 AM    <DIR>          GpMessages
10/14/2022  12:39 AM    <DIR>          ImportLog
10/12/2022  12:02 PM    <DIR>          Index
10/14/2022  01:01 AM           158,530 inundation_.aprx
10/12/2022  11:01 AM               403 inundation_.atbx
10/19/2022  01:52 PM    <DIR>          inundation_.gdb
10/19/2022  01:28 PM            16,688 lab-03-inundation.ipynb
10/12/2022  02:54 PM    <DIR>          RasterFunctionTemplates
10/14/2022  12:39 AM            27,439 Scene2.pdf
               4 File(s)        203,060 bytes
               9 Dir(s)  67,406,311,424 bytes free

Feature classes in geodatabase ['NHDFlowline_tc', 'candidate', 'candidate_unsplit', 'river_buffer', 'river_pts', 'rive

In [11]:
######### MODIFY THIS CELL: What stream do you want to analyze? ######### 
## DOT RUN AFTER EDITING MY STREAM!!!

# Search for a stream by name. 
selectRiver('Troublesome')

Using "GNIS_Name" like '%Troublesome%' to match all stream names that include Troublesome...
Edit the section of the stream that you want to create the REM. 
Then select that section using the Select tool.
Finished processing 2022-10-13 13:04:45.577540


### Edit the stream flowline

Make the flowline follow the stream as observed from the aerial photography used in our first lab. The tighter the fit, the better the results.

![Stream flowline not aligned](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/1.jpg)   
*Stream flowline not aligned*

![Edit stream flowline](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/2.jpg)   
*Edit stream flowline*

![Edit and save stream layer](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/3.jpg)   
*Edit and save stream layer*


![Select contiguous stream segment](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/4.jpg)   
*Select contiguous stream segment*

In [5]:
######### MODIFY THIS CELL: What parameters do you want to use to analyze your stream? ######### 

# Process stream to generate points and create buffer polygon
# 1st parameter: distance in feet between river sample points
# 2nd paramter: buffer distance from flowline in feet

# NOTE: Before you run this, make a contiguous selection using the Select Tool to test the results.
processRiver(40, 900)

Do you have a selection to process? Yes or No Yes
AOI will be 900 feet from flowline. Placing points every 40 feet along flowline...
Finished processing 2022-10-19 13:56:59.780623


![Points used to sample river elevation](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/4.1.jpg)   
*Points used to sample river elevation*

### Relative elevation model

This step creates the surface that interpolates our selected stream's elevation change as it flows down its watershed. This surface will allow us to find adjacent area's elevation relative to the stream's elevation.


![Example output classified](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/5.jpg)   
*Example output symbolized in classes*

![Example output classified](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/6.jpg)   
*Example output symbolized as gradient*


In [6]:
######### MODIFY THIS CELL: Use the buffer distance applied in the above function. ######### 

# Create REM
# Single parameter: buffer distance from line in feet. In most cases should match the buffer distance used above.
processREM(900)

Clipping DEM to AOI...
Sampling ground elevation along flowline...
Interpolating stream height...
Calculating REM on ground surface...
Calculating REM on above ground surface...
Finished processing 2022-10-19 14:16:00.376123


### Inundation model

If flood waters reach a certain height above the stream, what areas would be inundated?


![Example output classified](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/7.jpg)   
*inundation over bare earth*

![Example output classified](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/8.jpg)   
*inundation over bare earth with DSM*

![Example output classified](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/9.jpg)   
*Inundation over abover ground features with DSM*



In [7]:
######### MODIFY THIS CELL: What height above the stream do want for inundation? ######### 

# Create inundation model
# Single parameter: height above stream in feet
processInundation(25)  

Calculating extent of innudation to 25 feet above stream level for ground...
Calculating extent of innudation to 25 feet above stream level for above ground features...
Finished processing 2022-10-19 14:22:12.098122


In [31]:
######### DO NOT MODIFY THIS CELL ######### 

# Create an aerial image for the 2019 6in color imagery to augment visualization
print('Clipping aerial imagery to AOI...')
arcpy.management.Clip(img, "#", 'img', 'river_buffer', '#', True)

Clipping aerial imagery to AOI...


<Result 'C:\\BoydsGIS\\inundation_\\inundation_.gdb\\img'>

In [28]:
c = arcpy.sa.ExtractByMask(arcpy.sa.Raster(dsm), 'river_buffer')
s = arcpy.sa.Con(arcpy.sa.IsNull(arcpy.sa.Raster('inun_dsm_25ft')),   c, arcpy.sa.Raster('inun_dsm_25ft') + c)
# surface = arcpy.sa.Raster('dsm') + arcpy.sa.Raster('inun_dsm_25ft')

In [29]:
s.save('surface2')

In [30]:
c = arcpy.sa.ExtractByMask(arcpy.sa.Raster(dem), 'river_buffer')
s = arcpy.sa.Con(arcpy.sa.IsNull(arcpy.sa.Raster('inun_dem_25ft')),   c, arcpy.sa.Raster('inun_dem_25ft') + c)
# surface = arcpy.sa.Raster('dsm') + arcpy.sa.Raster('inun_dsm_25ft')
s.save('surface3')

### Using aerial imagery

This cell will extract 2019 aerial imagery to help visualize and identify inundated areas.


![Example output classified](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/10.jpg)   
**

![Example output classified](https://nyc3.digitaloceanspaces.com/astoria/geo509/lab-03/images/11.jpg)   
**



![](HAR_Elkhorn.jpg)